In [ ]:
! pip install BeautifulSoup4 pandas selenium

In [ ]:
! pip install webdriver-manager


🧩 Étape 2 — Importer les librairies et lancer le navigateur


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


In [5]:

# 🚀 ChromeDriver téléchargé automatiquement
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


In [ ]:


# 🌍 Ouvre la page FBref
driver.get("https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats")
time.sleep(5)

# 🔍 Récupère le tableau Premier League
table = driver.find_element(By.ID, "results2024-202591_overall")
rows = table.find_elements(By.TAG_NAME, "tr")

headers = [th.text for th in rows[0].find_elements(By.TAG_NAME, "th")]
data = []

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) > 0:
        data.append([col.text for col in cols])

# 💾 Sauvegarde dans un CSV
df = pd.DataFrame(data, columns=headers[1:])
df.to_csv("premier_league_table.csv", index=False)

print("✅ Tableau enregistré dans premier_league_table.csv")

driver.quit()




✅ Tableau enregistré dans premier_league_table.csv
